In [ ]:
#Preprosess the dataset
#Same process as performed in the project_report3.ipynb file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
import time
import re
import nltk.corpus
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from keras import layers
from keras import initializers as initializers, regularizers, constraints
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.models import Model,Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.preprocessing.text import Tokenizer

from keras.optimizers import Adam
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
url = 'https://raw.githubusercontent.com/dashascience/-MBTI-Myers-Briggs-Personality-Type-Dataset/master/mbti_1.csv'
df = pd.read_csv(url)

In [ ]:
# normalize, remove stop words and url links.
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
clean_text1=[]
for i in df.posts[0].split('|||'):
  i=i.lower()
  i = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", i)
  i_tokens = word_tokenize(i)
  filtered_i = [w for w in i_tokens if not w in stop_words]
  clean_text1.append(filtered_i)

In [ ]:
#lemmatization ensure that only words that mean similar things are consolidated into a single token
for j in clean_text1:
  for p in range(len(j)):
    j[p]=lemmatizer.lemmatize(j[p])


In [ ]:
# Clean methods for dataframe cleaning
def clean_data(str):
  clean_text1=[]
  for i in str.split('|||'):
    i=i.lower()
    i = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", i)
    i_tokens = word_tokenize(i)
    filtered_i = [w for w in i_tokens if not w in stop_words]
    clean_text1.append(filtered_i)
  return clean_text1

def lemmatized_data(strList):
  for j in strList:
    for p in range(len(j)):
      j[p]=lemmatizer.lemmatize(j[p])
  return strList

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
#MBTI related words
mbti = ["intj","intp","infj", "infp","istj","istp","isfj","isfp","entj","entp", "enfj", "enfp","estj", "estp","esfj","esfp","si","ni","ti","fi","se",
        "ne","te","fe","nt","nf","le","mbti","sp","sj","sf","st","in","is","en","es"]
data_length=[]
lemmatizer=WordNetLemmatizer()
clean_text1=[]
for i in range(0,len(df.posts)):
  doc = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", df.posts[i])
  doc = doc.lower()
  doc = doc.split()
  #Remove MBTI related words and lemmatize the rest words
  for w in doc:
    if w in mbti:
      doc.remove(w)
    elif not w in set(stopwords):
        lemmatizer.lemmatize(w) 

      
  doc = ' '.join(doc)
  clean_text1.append(doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
types = np.unique(df.type.values)

def get_type_index(string):
    return list(types).index(string)

df['type_index'] = df['type'].apply(get_type_index)
# Find the original posts column
n = df.columns[1]
# Drop that column
df.drop(n, axis = 1, inplace = True)
# Put cleaned posts in place
df[n] = clean_text1

In [ ]:
df

,type,type_index,posts
0,INFJ,8,and moments sportscenter not top ten plays pra...
1,ENTP,3,im finding the lack of me these posts very ala...
2,INTP,11,good one course to which i say i know thats my...
3,INTJ,10,dear i enjoyed our conversation the other day ...
4,ENTJ,2,youre firedthats another silly misconception t...
...,...,...,...
8670,ISFP,13,just because i always think of cats as doms fo...
8671,ENFP,1,soif this thread already exists someplace else...
8672,INTP,11,so many questions when i do these things i wou...
8673,INFP,9,i am very conflicted right now when it comes t...


In [ ]:
#Shows posts by group, we can see that the distribution is unbalanced
#ESTJ has the lowest number of posts which is 39
print(df.groupby(['type']).count().mean())
print(df.groupby(['type']).count())

type_index    542.1875
posts         542.1875
dtype: float64
      type_index  posts
type                   
ENFJ         190    190
ENFP         675    675
ENTJ         231    231
ENTP         685    685
ESFJ          42     42
ESFP          48     48
ESTJ          39     39
ESTP          89     89
INFJ        1470   1470
INFP        1832   1832
INTJ        1091   1091
INTP        1304   1304
ISFJ         166    166
ISFP         271    271
ISTJ         205    205
ISTP         337    337


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#specify the amount of features
vocab_size = 10000

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(df.posts)

import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=39) # fit predictor and target variable


maxlen = 500
#Transform text into sequence and pad posts with size of 500
train_sequences = tokenizer.texts_to_sequences(df.posts.values)
train_padded = pad_sequences(train_sequences, maxlen = maxlen)
labels= tf.keras.utils.to_categorical(df.type_index.values, num_classes=16)
x_rus, y_rus = ros.fit_resample(train_padded, labels)
x_train,x_test,y_train,y_test=train_test_split(x_rus, y_rus, test_size=0.2, random_state=42)

#Transform text into sequence and pad posts with size of 500 for validation set

#show shape of different sets                            
print("Training set shape:",x_train.shape)
print("Testing set shape:",x_test.shape)


Training set shape: (23449, 500)
Testing set shape: (5863, 500)


In [ ]:
#Apply grid search to fine-tune hyper-parameters
from tensorflow.keras.layers import MaxPooling1D,Conv1D,Dense,Conv2D,MaxPool2D,Flatten,ReLU,Dropout,Flatten
def create_model_CNN(lrn_rate,drop_rate):
    model = Sequential()
    model.add(Embedding(vocab_size,64,input_length=len(train_padded[0])))

    model.add(Conv1D(64,5,activation = "relu"))
    model.add(Dropout(drop_rate))
    model.add(MaxPooling1D())

    model.add(Conv1D(64,5,activation = "relu"))
    model.add(Dropout(drop_rate))
    #flatten the data for the softmax layer
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(drop_rate))
    model.add(Dense(16, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer = tf.optimizers.Adam(learning_rate=lrn_rate), metrics=['accuracy'])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

model = KerasClassifier(build_fn= create_model_CNN)

#Apply hyperparameter tunning using a Grid search
#Hyper parameters used in the grid search process: batch_size, epoches, learning rate, and Dropout rate
params= {'batch_size':[60,350], 
         'epochs':[40,60],
        'lrn_rate':[0.001,0.020],
        'drop_rate':[0.15,0.2,0.1],
        }

#Cross validation 
gs=GridSearchCV(estimator=model, param_grid=params,cv=3)

result = gs.fit(x_train, y_train)

<ipython-input-12-f06bec95e26d>:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn= create_model_CNN)


Streaming output truncated to the last 5000 lines.
261/261 [==============================] - 2s 9ms/step - loss: 2.7748 - accuracy: 0.0661
Epoch 26/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7754 - accuracy: 0.0620
Epoch 27/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7750 - accuracy: 0.0626
Epoch 28/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7756 - accuracy: 0.0611
Epoch 29/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7755 - accuracy: 0.0606
Epoch 30/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7759 - accuracy: 0.0613
Epoch 31/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7754 - accuracy: 0.0592
Epoch 32/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7748 - accuracy: 0.0667
Epoch 33/60
261/261 [==============================] - 2s 9ms/step - loss: 2.7753 - accuracy: 0.0633
Epoch 34/60
261/261 [==============================]

In [ ]:
best_params=result.best_params_
accuracy=result.best_score_
#Print the best combination of hyper parameters
print("Best result in Grid search: Accuracy %f using %s" % (result.best_score_, result.best_params_))

Best result in Grid search: Accuracy 0.823276 using {'batch_size': 350, 'drop_rate': 0.1, 'epochs': 40, 'lrn_rate': 0.001}
